# Домашнее задание: BiLSTM для задачи PoS Tagging

В этом ноутбуке мы будем создавать модель машинного обучения, которая генерирует результат для каждого элемента входной последовательности с использованием PyTorch и TorchText. Конкретно, мы будем подавать текст на вход, а модель будет выводить метку - часть речи (PoS) для каждого токена во входном тексте. Этот подход также может применяться для распознавания именованных сущностей (NER), где результатом для каждого токена будет указание на тип сущности, если таковая имеется.

В этом блокноте мы реализуем многослойную двунаправленную LSTM (BiLSTM) для предсказания меток частей речи с использованием набора данных Universal Dependencies English Web Treebank (UDPOS).

In [ ]:
!pip install torchtext==0.6.0

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim

from torchtext import data
from torchtext import datasets

import spacy
import numpy as np

import time
import random

Зафиксируем случайности для воспроизводимости результатов.

In [ ]:
SEED = 1234

random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

В этом наборе данных есть два разных набора меток: метки универсальных зависимостей (UD) и метки Penn Treebank (PTB). Мы будем обучать модель только на метках UD, но загрузим метки PTB, чтобы показать, как их можно использовать вместо них.

* UD_TAGS определяет, как следует обрабатывать метки UD. В нашем словаре TEXT, который мы создадим позже, будут неизвестные токены, то есть токены, которых нет в нашем словаре. Однако у нас не будет неизвестных меток, поскольку мы имеем дело с конечным набором возможных меток. Мы будем обозначать неизвестные токены как <unk>, и затем будем их убирать, установив unk_token = None.

* PTB_TAGS выполняет то же самое, что и UD_TAGS, но обрабатывает метки PTB.

In [ ]:
from torchtext.data import Field

TEXT = Field(lower = True)
UD_TAGS = Field(unk_token = None)
PTB_TAGS = Field(unk_token = None)

In [ ]:
fields = (("text", TEXT), ("udtags", UD_TAGS), ("ptbtags", PTB_TAGS))

Загрузим датасет UDPOS.

In [ ]:
train_data, valid_data, test_data = datasets.UDPOS.splits(fields)

## Задание

Посмотрите на количество объектов в датасетах `train_data, valid_data и test_data`. В ответ запишите число объектов в самом маленьком датасете.

In [ ]:
# ваш код здесь

Напечатаем пример из датасета

In [ ]:
print(vars(train_data.examples[0]))

Можем отдельно посмотреть на текст и на теги

In [ ]:
print(vars(train_data.examples[0])['text'])

In [ ]:
print(vars(train_data.examples[0])['udtags'])

In [ ]:
print(vars(train_data.examples[0])['ptbtags'])

Что мы сделаем дальше:

* Мы создадим словарь - отображение токенов в целые числа.

* Мы хотим, чтобы в нашем наборе данных были некоторые неизвестные токены, чтобы воссоздать, как эта модель будет использоваться в реальной жизни, поэтому мы устанавливаем `min_freq = 2`, что означает, что в словарь будут добавлены только токены, появляющиеся хотя бы дважды в обучающем наборе, и остальные будут заменены токенами `<unk>`.

* Мы также загружаем предобученные векторы GloVe длины 100 для инициализации эмбеддингов.

* `unk_init` используется для инициализации эмбеддингов токенов, которых нет в словаре предварительно обученных вложений. По умолчанию эта инициализация устанавливает эти эмбеддинги в нули, однако лучше избежать их инициализации одним и тем же значением, поэтому мы инициализируем их из нормального распределения.

* Предобученные векторы загружаем в наш словарь и будем инициализировать нашу модель этими значениями позже.

## Задание

По тренировочным данным постройте три словаря, используя `build_vocab`:

* Cловарь по текстам `TEXT` с гиперпараметрами:
  * min_freq = MIN_FREQ
  * vectors = "glove.6B.100d"
  * unk_init = torch.Tensor.normal_

* Словарь по `UD_TAGS`

* Словарь по `PTB_TAGS`

Сколько уникальных токенов в словаре, построенном по текстам?

In [ ]:
MIN_FREQ = 2

# ваш код здесь

## Задание

Какой самый популярный (часто встречающийся) токен в словаре, построенном по текстам?

In [ ]:
# ваш код здесь

Посмотрим на функцию, вычисляющую процентное соотношение тегов в текстах.

In [ ]:
def tag_percentage(tag_counts):

    total_count = sum([count for tag, count in tag_counts])

    tag_counts_percentages = [(tag, count, count/total_count) for tag, count in tag_counts]

    return tag_counts_percentages

## Задание

Пользуясь функцией `tag_percentage`, выведите на экран процентное соотношение каждого UD-тэга.

Какой тег встречается в текстах чаще всего (в процентах)?

In [ ]:
# ваш код здесь

## Задание

Используя `BucketIterator.split`, создайте объекты `train_iterator, valid_iterator, test_iterator` для итерирования по батчам.

In [ ]:
BATCH_SIZE = 128

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

train_iterator, valid_iterator, test_iterator = # ваш код здесь

## Создаем архитектуру нейронной сети

![](https://github.com/bentrevett/pytorch-pos-tagging/blob/master/assets/pos-bidirectional-lstm.png?raw=1)

Задайте нейронную сеть по аналогии с сетью из вебинара:

* Слой Embedding:
  * помимо прочего задайте `padding_idx = pad_idx`

* Затем слой LSTM с гиперпараметрами:
  * `n_layers = 1`
  * `bidirectional = True`
  * задайте `dropout`

* Затем DropOut слой

* Линейный слой, принимающий на вход `hidden_dim * 2` нейронов (так как двунаправленная сеть) и на выходе `output_dim` нейронов

В ответ на задание выберите, как выглядит первая строчка в архитектуре сети.

In [ ]:
class BiLSTMPOSTagger(nn.Module):
    def __init__(self,
                 input_dim,
                 embedding_dim,
                 hidden_dim,
                 output_dim,
                 n_layers,
                 bidirectional,
                 dropout,
                 pad_idx):

        super().__init__()

        # ваш код здесь


    def forward(self, text):

        #pass text through embedding layer and then through dropout layer
        embedded = # ваш код здесь

        #pass embeddings into LSTM
        outputs, (hidden, cell) = # ваш код здесь

        #apply dropout and then linear layer
        predictions = # ваш код здесь

        return predictions

## Обучение модели

## Задание

Запустите ячейку ниже. Если класс `BiLSTMPOSTagger` реализован корректно, ячейка отработает без ошибок. Получилось?

In [ ]:
INPUT_DIM = len(TEXT.vocab)
EMBEDDING_DIM = 100
HIDDEN_DIM = 128
OUTPUT_DIM = len(UD_TAGS.vocab)
N_LAYERS = 2
BIDIRECTIONAL = True
DROPOUT = 0.25
PAD_IDX = TEXT.vocab.stoi[TEXT.pad_token]

model = BiLSTMPOSTagger(INPUT_DIM,
                        EMBEDDING_DIM,
                        HIDDEN_DIM,
                        OUTPUT_DIM,
                        N_LAYERS,
                        BIDIRECTIONAL,
                        DROPOUT,
                        PAD_IDX)

Инициализируем веса сети числами из стандартного нормального распределения

In [ ]:
def init_weights(m):
    for name, param in m.named_parameters():
        nn.init.normal_(param.data, mean = 0, std = 0.1)

model.apply(init_weights)

## Задание

Напишите функцию для вычисления количества весов сети.

С помощью этой функции выведите на экран число весов нашей сети.

In [ ]:
def count_parameters(model):
    # ваш код здесь

# ваш код здесь

## Задание

Инициализируйте embedding-слой сети предобученными GloVe-векторами.

В ответ напишите число координат в предобученных эмбеддингах.

In [ ]:
# ваш код здесь

Инициализируем нулями pad-токены

In [ ]:
model.embedding.weight.data[PAD_IDX] = torch.zeros(EMBEDDING_DIM)

print(model.embedding.weight.data)

Зададим оптимизатор

In [ ]:
optimizer = optim.Adam(model.parameters())

Зададим loss.

В случае токена `<pad>` (пустота) лосс мы не считаем, поэтому индексы таких токенов мы пропускаем (игнорируем).

In [ ]:
TAG_PAD_IDX = UD_TAGS.vocab.stoi[UD_TAGS.pad_token]

criterion = nn.CrossEntropyLoss(ignore_index = TAG_PAD_IDX)

Переносим модель на GPU по возможности

In [ ]:
model = model.to(device)
criterion = criterion.to(device)

Функция ниже вычисляет `accuracy` для каждого батча

In [ ]:
def categorical_accuracy(preds, y, tag_pad_idx):
    """
    Returns accuracy per batch, i.e. if you get 8/10 right, this returns 0.8, NOT 8
    """
    max_preds = preds.argmax(dim = 1, keepdim = True) # get the index of the max probability
    non_pad_elements = (y != tag_pad_idx).nonzero()
    correct = max_preds[non_pad_elements].squeeze(1).eq(y[non_pad_elements])
    return correct.sum() / y[non_pad_elements].shape[0]

## Задание

Допишите цикл обучения модели.

Для каждого батча на каждой итерации:
- зануляем градиенты
- применяем модель к батчу
- делаем reshape прогнозов, так как loss нельзя вычислить для тензора размерности 3 (это уже написано)
- вычисляем loss и accuracy
- вычисляем градиенты и делаем шаг градиентного спуска

In [ ]:
def train(model, iterator, optimizer, criterion, tag_pad_idx):

    epoch_loss = 0
    epoch_acc = 0

    model.train()

    for batch in iterator:

        # ваш код здесь

        predictions = predictions.view(-1, predictions.shape[-1]) # predictions - прогнозы модели
        tags = tags.view(-1) # tags - правильные ответы (метки)

        # ваш код здесь

    return epoch_loss / len(iterator), epoch_acc / len(iterator)

Функцию `evaluate` для простоты мы написали.

In [ ]:
def evaluate(model, iterator, criterion, tag_pad_idx):

    epoch_loss = 0
    epoch_acc = 0

    model.eval()

    with torch.no_grad():

        for batch in iterator:

            text = batch.text
            tags = batch.udtags

            predictions = model(text)

            predictions = predictions.view(-1, predictions.shape[-1])
            tags = tags.view(-1)

            loss = criterion(predictions, tags)

            acc = categorical_accuracy(predictions, tags, tag_pad_idx)

            epoch_loss += loss.item()
            epoch_acc += acc.item()

    return epoch_loss / len(iterator), epoch_acc / len(iterator)

Ниже функция, которая замеряет время обучения на каждой эпохе

In [ ]:
def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

## Задание

Обучим нашу модель. Допишите цикл по подсказкам в коде.

Какая accuracy (в процентах) получается на валидации на последней эпохе? Ответ округлите до целого числа.

In [ ]:
N_EPOCHS = 10

best_valid_loss = float('inf')

for epoch in range(N_EPOCHS):

    start_time = time.time()

    train_loss, train_acc = # ваш код здесь - примените функцию для обучения модели
    valid_loss, valid_acc = # ваш код здесь - примените функцию для применения и оценки качества модели

    end_time = time.time()

    epoch_mins, epoch_secs = # замерьте время выполнения эпохи, используя написанную для этого функцию

    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'tut1-model.pt')


    # для каждой эпохи выведите train loss, train accuracy, val loss, val accuracy, epoch time
    # ваш код здесь

Посмотрим на качество обученной модели на тесте

In [ ]:
model.load_state_dict(torch.load('tut1-model.pt'))

test_loss, test_acc = evaluate(model, test_iterator, criterion, TAG_PAD_IDX)

print(f'Test Loss: {test_loss:.3f} |  Test Acc: {test_acc*100:.2f}%')

## Инференс

Посмотрим, как модель работает на новых данных. Допишите функцию `tag_sentence` для применения обученной модели, по подсказкам ниже.

In [ ]:
def tag_sentence(model, device, sentence, text_field, tag_field):

    # ваш код здесь - переведите модель в режим применения

    if isinstance(sentence, str):
        nlp = spacy.load('en_core_web_sm')
        tokens = [token.text for token in nlp(sentence)]
    else:
        tokens = [token for token in sentence]

    if text_field.lower:
        tokens = [t.lower() for t in tokens]

    numericalized_tokens = # ваш код здесь - создайте список, состоящий из переведенных в индексы токенов из словаря text_field.vocab (используйте stoi)

    unk_idx = text_field.vocab.stoi[text_field.unk_token]

    unks = [t for t, n in zip(tokens, numericalized_tokens) if n == unk_idx]

    token_tensor = # ваш код здесь - приведите numericalized_tokens к типу torch.LongTensor

    token_tensor = token_tensor.unsqueeze(-1).to(device)

    predictions = # ваш код здесь - примените модель к token_tenzor

    top_predictions = predictions.argmax(-1)

    predicted_tags = [tag_field.vocab.itos[t.item()] for t in top_predictions]

    return tokens, predicted_tags, unks

## Задание

Запустите две следующие ячейки. Проверим, что написанная функция работает корректно.

В ответе выберите те токены, которые были нераспознаны (их не было в обучающих данных).

In [ ]:
example_index = 1

sentence = vars(train_data.examples[example_index])['text']
actual_tags = vars(train_data.examples[example_index])['udtags']

print(sentence)

In [ ]:
tokens, pred_tags, unks = tag_sentence(model,
                                       device,
                                       sentence,
                                       TEXT,
                                       UD_TAGS)

print(unks)

## Задание

Проверим качество модели. Запустите ячейку ниже. В ответе укажите число неверно классифицированных токенов.

In [ ]:
print("Pred. Tag\tActual Tag\tCorrect?\tToken\n")

for token, pred_tag, actual_tag in zip(tokens, pred_tags, actual_tags):
    correct = '✔' if pred_tag == actual_tag else '✘'
    print(f"{pred_tag}\t\t{actual_tag}\t\t{correct}\t\t{token}")

## Задание

Примените модель к любому предложению (на английском языке). Какая доля токенов размечена верно? Ответ напишите в комментариях.

In [ ]:
# ваш код здесь